In [1]:
from imdb import Cinemagoer

In [2]:
ia = Cinemagoer()

results = ia.search_movie('the matrix')

In [3]:
results

[<Movie id:0133093[http] title:_The Matrix (1999)_>,
 <Movie id:10838180[http] title:_The Matrix Resurrections (2021)_>,
 <Movie id:0234215[http] title:_The Matrix Reloaded (2003)_>,
 <Movie id:0242653[http] title:_The Matrix Revolutions (2003)_>,
 <Movie id:0410519[http] title:_The Matrix Recalibrated (2004) (V)_>,
 <Movie id:11574780[http] title:_The Matrix Redeemed (2020)_>,
 <Movie id:9847360[http] title:_A Glitch in the Matrix (2021)_>,
 <Movie id:12355912[http] title:_The Matrix: Reborn (2020) (TV)_>,
 <Movie id:0274085[http] title:_Sex and the Matrix (2000)_>,
 <Movie id:30749809[http] title:_Free Your Mind: The Matrix Now (2023)_>,
 <Movie id:0295432[http] title:_The Matrix Revisited (2001) (V)_>,
 <Movie id:0365467[http] title:_Making 'The Matrix' (1999)_>,
 <Movie id:0106062[http] title:_"Matrix" (1993)_>,
 <Movie id:9642498[http] title:_The Matrix (2016)_>,
 <Movie id:31998838[http] title:_Matrix 5 (None)_>,
 <Movie id:0277828[http] title:_Enter the Matrix (2003) (VG)_>,
 <M

In [4]:
# Create an instance of the IMDb class


# Get movie by IMDb ID
movie_id = '6723592'  # This is the IMDb ID for the movie "Interstellar"
movie = ia.get_movie(movie_id)

movie

<Movie id:6723592[http] title:_Tenet (2020)_>

In [6]:
genre = movie.get('genre')

In [15]:
# Get the first cast member of the movie
first_cast_member = movie['cast'][4] if 'cast' in movie else None

In [16]:
first_cast_member

<Person id:0913475[http] name:_John David Washington_>

In [17]:
if first_cast_member:
    actor_id = first_cast_member.personID
    actor = ia.get_person(actor_id)  # Retrieve full details of the actor
    
    # Extracting various pieces of information
    actor_name = actor.get('name', 'N/A')
    actor_birth_date = actor.get('birth date', 'N/A')
    actor_height = actor.get('height', 'N/A')
    actor_biography = actor.get('biography', ['N/A'])[0]  # IMDb stores biography as a list
    actor_filmography = actor.get('filmography', 'N/A')  # Detailed filmography
    actor_trivia = actor.get('trivia', ['N/A'])[0]  # Trivia is also a list, taking the first entry
    
    actor_photo_url = actor.get('headshot', 'No photo available')
    
    # Print all available details
    print(f"Actor Name: {actor_name}")
    print(f"Birth Date: {actor_birth_date}")
    print(f"Height: {actor_height}")
    print(f"Biography: {actor_biography[:500]}...")  # Limiting biography output
    print(f"Trivia: {actor_trivia}")
    print(f"Filmography: {actor_filmography}")
    print(f"Actor Photo URL: {actor_photo_url}")
else:
    print("No cast member found.")

Actor Name: John David Washington
Birth Date: N/A
Height: N/A
Biography: John David Washington is an American actor, producer, and former professional football player. He played college football at Morehouse College and signed with the St. Louis Rams as an Undrafted free agent in 2006. Professionally, Washington spent four years as the running back for the United Football League's Sacramento Mountain Lions. Washington shifted to an acting career like his father, 'Denzel Washington' (qv), and mother, Pauletta. He was part of the main cast of the HBO comedy series Ball...
Trivia: N/A
Filmography: N/A
Actor Photo URL: https://m.media-amazon.com/images/M/MV5BOTY4NDcyNDM5OF5BMl5BanBnXkFtZTgwMjk4Mzk0NTM@._V1_QL75_UY207_CR4,0,140,207_.jpg


In [48]:
print(actor_biography)

Jessica Michelle Chastain was born in Sacramento, California, and was raised in a middle-class household in a Northern California suburb. Her mother, Jerri Chastain, is a vegan chef whose family is originally from Kansas, and her stepfather is a fireman. She discovered dance at the age of nine and was in a dance troupe by age thirteen. She began performing in Shakespearean productions all over the Bay area.An actor in a production of "Romeo & Juliet" encouraged her to audition for Juilliard as a drama major. She became a member of "Crew 32" with the help of a scholarship from one of the school's famous alumni, 'Robin Williams' (qv).In her last year at Juilliard, she was offered a holding deal with TV writer/producer 'John Wells' (qv) and she eventually worked in three of his TV shows. Jessica continues to do theatre, having played in "The Cherry Orchard", "Rodney's Wife", "Salome" and "Othello". She spends her time between New York and Los Angeles, working in theater, film and TV.In 20

In [58]:
import pprint

actor = ia.get_person(first_cast_member.personID)

print(actor.get('filmography'))

None


In [ ]:
import os
import openai
import json
from dotenv import load_dotenv
import pymongo
from bson.objectid import ObjectId

load_dotenv() 
mongodb_uri = os.getenv('MONGODB_URI')  # retrieve mongodb uri from .env file

try:
    client = pymongo.MongoClient(mongodb_uri)  # this creates a client that can connect to our DB
    db = client.get_database("Movies")  # this gets the database named 'Movies'
    movieDetails = db.get_collection("MovieDetail")

    client.server_info()  # forces client to connect to server
    print("Connected successfully to the 'Movies' database!")

except pymongo.errors.ConnectionFailure as e:
    print(f"Could not connect to MongoDB: {e}")
    exit(1)

def get_movie_poster_details(poster_link):
    # Structured prompt to enforce the same parameters for all movie posters
    prompt = f"Provide the following information about the poster {poster_link} as JSON:\n\
    title,\n\
    tagline,\n\
    genre,\n\
    director_style,\n\
    color_palette (nested object containing HEX codes of primary, secondary, and accent colors),\n\
    font (nested object containing title_font, tagline_font, credits_font),\n\
    image_elements (e.g., main character, background),\n\
    atmosphere,\n\
    iconography,\n\
    art_style,\n\
    period_style.\n\
    If any information is unavailable, use 'unknown' as the value."
    
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=300
    )
    
    # Parse GPT response to JSON
    details = json.loads(response['choices'][0]['text'].strip())
    
    return details

def add_movie_details():  # defines what information we are looking to store
    imdb_id = "Sample id"
    Title = "Sample Title"
    Rating = "Sample Rating"
    RTM = "Sample RTM"
    DOR = "Sample DOR"
    Genre = ["Genre1", "Genre2", "Genre3"]  # store multiple genres to calculate similarity score
    Director = "Sample Director"
    Writers = ["Writer1", "Writer2", "Writer3"]
    Actors = [
        {
            "Actor 1": "Sample Actor1",
            "Actor 2": "Sample Actor 2",
            "Actor 3": "Sample Actor 3"
        }
    ]
    Plot = "Sample Plot goes here."
    Estimated_Budget = "Sample Budget"
    Poster = "Sample Poster Link"

    # Call the GPT function to get the Poster Key Characteristics
    poster_details = get_movie_poster_details(Poster)

    # Construct poster_document with default values for missing fields
    poster_document = {
        "title": poster_details.get("title", "unknown"),
        "tagline": poster_details.get("tagline", "unknown"),
        "genre": poster_details.get("genre", "unknown"),
        "director_style": poster_details.get("director_style", "unknown"),
        "color_palette": poster_details.get("color_palette", {
            "primary": "unknown",
            "secondary": "unknown",
            "accent": "unknown"
        }),
        "font": poster_details.get("font", {
            "title_font": "unknown",
            "tagline_font": "unknown",
            "credits_font": "unknown"
        }),
        "image_elements": poster_details.get("image_elements", {
            "main_character": "unknown",
            "background": "unknown"
        }),
        "atmosphere": poster_details.get("atmosphere", "unknown"),
        "iconography": poster_details.get("iconography", ["unknown"]),
        "art_style": poster_details.get("art_style", "unknown"),
        "period_style": poster_details.get("period_style", "unknown")
    }

    # Full movie detail, now embedding Poster_Key_Characteristics retrieved from GPT
    movieDetail = {
        "imdb_id": imdb_id,
        "title": Title,
        "Rating": Rating,
        "RTM": RTM,
        "DOR": DOR,
        "Genre": Genre,
        "Director": Director,
        "Writer(s)": Writers,
        "Actors": Actors,
        "Plot": Plot,
        "Estimated Budget": Estimated_Budget,
        "Poster": Poster,
        "Poster_Key_Characteristics": poster_document  # Embedding GPT-generated poster characteristics
    }

    try:
        result = movieDetails.insert_one(movieDetail)
        print(f"Movie added with ID: {result.inserted_id}")
    except pymongo.errors.PyMongoError as e:
        print(f"An error occurred while adding the movie: {e}")

def main():
    add_movie_details()
    client.close()

if __name__ == "__main__":
    main()
